<a href="https://colab.research.google.com/github/Vandan1009/python_1/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
!pip install pyspark

In [133]:
from pyspark.sql import SparkSession

In [134]:
spark=SparkSession.builder.appName('project1').getOrCreate()

In [135]:
spark

In [137]:
df=spark.read.csv('/content/final_data.csv',header=True,inferSchema=True)

In [139]:
df1=spark.read.csv('/content/order_details_new.csv',header=True,inferSchema=True)

In [140]:
from pyspark.sql.functions import *

In [142]:
df.show()

+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|      product_name| Price|         Description|Category|StockQuantity|       Brand|Rating|         Warehouse|
+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|         Card Game|363.33|Whom those term b...|    Toys|           95|        Nerf|   2.6|        Emilyburgh|
|          Toy Boat|  79.6|Listen economy ge...|    Toys|           34|      Barbie|   2.8|  North Jamesmouth|
|     Action Figure|432.64|Lead back her local.|    Toys|            8|        Nerf|   4.6|  West Joshuahaven|
|      Rubik's Cube|215.39|Watch growth inve...|    Toys|         NULL|      Barbie|   3.5|   Mckenziechester|
|       Barbie Doll|  NULL|                NULL|    Toys|           38|   DC Comics|   1.2|     Catherineside|
|    Stuffed Animal|220.09|Computer little f...|    Toys|           76|      Chicco|   2.3|         Bryanport|
|

In [143]:
df1.show()

+--------+-----------+------------------+----------+--------------------+--------+----------+------------+--------------------+
|order_id|customer_id|     customer_name|product_id|      customer_email|quantity|order_date|order_status|    shipping_address|
+--------+-----------+------------------+----------+--------------------+--------+----------+------------+--------------------+
|      37|         20|    Kristy Flowers|        14|pamela11@example.com|       6|2024-07-06|     Pending|6479 Thomas Trail...|
|      80|         38|     Keith Sanchez|         6|lisaholt@example.org|       6|2024-08-10|     Shipped|07201 Berry Passa...|
|      52|         70|       Frank Myers|        30|  nyoung@example.net|       4|2024-01-17|     Shipped|Unit 8916 Box 262...|
|      39|         38|    Samantha Gomez|        26|suarezjennifer@ex...|       4|2024-04-17|     Shipped|8091 Sellers Ferr...|
|      87|         79|     Brian Blevins|        20|carterluis@exampl...|       4|2024-01-08|   Cancelle

In [145]:

df.withColumn("Price", when(col("Price").isNull(), avg_price).otherwise(col("Price"))).show()

+------------------+-----------------+--------------------+--------+-------------+------------+------+------------------+
|      product_name|            Price|         Description|Category|StockQuantity|       Brand|Rating|         Warehouse|
+------------------+-----------------+--------------------+--------+-------------+------------+------+------------------+
|         Card Game|           363.33|Whom those term b...|    Toys|           95|        Nerf|   2.6|        Emilyburgh|
|          Toy Boat|             79.6|Listen economy ge...|    Toys|           34|      Barbie|   2.8|  North Jamesmouth|
|     Action Figure|           432.64|Lead back her local.|    Toys|            8|        Nerf|   4.6|  West Joshuahaven|
|      Rubik's Cube|           215.39|Watch growth inve...|    Toys|         NULL|      Barbie|   3.5|   Mckenziechester|
|       Barbie Doll|10665.89385582563|                NULL|    Toys|           38|   DC Comics|   1.2|     Catherineside|
|    Stuffed Animal|    

In [144]:
df.selectExpr("avg(Price)").first()[0]

10665.89385582563

In [146]:
df.fillna({
    "product_name": "Unknown",
    "Description": "No description available",
    "StockQuantity": 0,
    "Brand": "No brand",
    "Rating": 0.0,
    "Warehouse": "Unknown"
}).show()

+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|      product_name| Price|         Description|Category|StockQuantity|       Brand|Rating|         Warehouse|
+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|         Card Game|363.33|Whom those term b...|    Toys|           95|        Nerf|   2.6|        Emilyburgh|
|          Toy Boat|  79.6|Listen economy ge...|    Toys|           34|      Barbie|   2.8|  North Jamesmouth|
|     Action Figure|432.64|Lead back her local.|    Toys|            8|        Nerf|   4.6|  West Joshuahaven|
|      Rubik's Cube|215.39|Watch growth inve...|    Toys|            0|      Barbie|   3.5|   Mckenziechester|
|       Barbie Doll|  NULL|No description av...|    Toys|           38|   DC Comics|   1.2|     Catherineside|
|    Stuffed Animal|220.09|Computer little f...|    Toys|           76|      Chicco|   2.3|         Bryanport|
|

In [148]:
df.withColumn("rating", when(col("rating") == 0, 3.5).otherwise(col("rating"))).show()

+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|      product_name| Price|         Description|Category|StockQuantity|       Brand|rating|         Warehouse|
+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|         Card Game|363.33|Whom those term b...|    Toys|           95|        Nerf|   2.6|        Emilyburgh|
|          Toy Boat|  79.6|Listen economy ge...|    Toys|           34|      Barbie|   2.8|  North Jamesmouth|
|     Action Figure|432.64|Lead back her local.|    Toys|            8|        Nerf|   4.6|  West Joshuahaven|
|      Rubik's Cube|215.39|Watch growth inve...|    Toys|         NULL|      Barbie|   3.5|   Mckenziechester|
|       Barbie Doll|  NULL|                NULL|    Toys|           38|   DC Comics|   1.2|     Catherineside|
|    Stuffed Animal|220.09|Computer little f...|    Toys|           76|      Chicco|   2.3|         Bryanport|
|

In [153]:
df.withColumn("Price", col("Price").cast("int")).show()

+------------------+-----+--------------------+--------+-------------+------------+------+------------------+
|      product_name|Price|         Description|Category|StockQuantity|       Brand|Rating|         Warehouse|
+------------------+-----+--------------------+--------+-------------+------------+------+------------------+
|         Card Game|  363|Whom those term b...|    Toys|           95|        Nerf|   2.6|        Emilyburgh|
|          Toy Boat|   79|Listen economy ge...|    Toys|           34|      Barbie|   2.8|  North Jamesmouth|
|     Action Figure|  432|Lead back her local.|    Toys|            8|        Nerf|   4.6|  West Joshuahaven|
|      Rubik's Cube|  215|Watch growth inve...|    Toys|         NULL|      Barbie|   3.5|   Mckenziechester|
|       Barbie Doll| NULL|                NULL|    Toys|           38|   DC Comics|   1.2|     Catherineside|
|    Stuffed Animal|  220|Computer little f...|    Toys|           76|      Chicco|   2.3|         Bryanport|
|Remote Co

In [161]:

orders_id=df1.dropDuplicates(["Order_id"])
orders_id.show()

+--------+-----------+--------------------+----------+--------------------+--------+----------+------------+--------------------+
|order_id|customer_id|       customer_name|product_id|      customer_email|quantity|order_date|order_status|    shipping_address|
+--------+-----------+--------------------+----------+--------------------+--------+----------+------------+--------------------+
|       1|         40|        Amy Espinoza|        55|codyjimenez@examp...|      10|2024-05-23|   Cancelled|777 Danielle Junc...|
|       2|         93|     Timothy Sanchez|         3|zjennings@example...|       7|2024-02-18|     Shipped|71690 Brian Freew...|
|       3|         25|       Maxwell Avila|        74|timothy59@example...|       3|2024-04-20|     Shipped|08937 Estes Valle...|
|       4|         11|      Danny Peterson|        50|hesterlinda@examp...|       8|2024-01-15|     Pending|28855 Anthony Pla...|
|       5|         28|        John Garrett|        66|haleydurham@examp...|       9|2024-0

In [162]:
orders_id=df1.dropDuplicates(["Product_id"])
orders_id.show()

+--------+-----------+----------------+----------+--------------------+--------+----------+------------+--------------------+
|order_id|customer_id|   customer_name|product_id|      customer_email|quantity|order_date|order_status|    shipping_address|
+--------+-----------+----------------+----------+--------------------+--------+----------+------------+--------------------+
|      76|          9|   Katie Johnson|         1|  odavis@example.com|       8|2024-04-29|     Shipped|41241 Wright Land...|
|      52|         28|  Hannah Cabrera|         2|deckeraaron@examp...|      10|2024-06-03|   Cancelled|494 Stephen Cente...|
|      17|         76|  Monica Montoya|         3|patriciapeck@exam...|       7|2024-03-01|   Delivered|9234 Harrington K...|
|      52|         68|     Mary Jordan|         4| laura21@example.org|       6|2024-06-17|   Delivered|Unit 6162 Box 415...|
|      12|         65|   Mary Mcknight|         5|carterelizabeth@e...|       6|2024-03-06|     Pending|79585 Logan Th

In [166]:
orders_df = df1.withColumn("Order_date", col("Order_date").cast("date"))
orders_df.show()

+--------+-----------+------------------+----------+--------------------+--------+----------+------------+--------------------+
|order_id|customer_id|     customer_name|product_id|      customer_email|quantity|Order_date|order_status|    shipping_address|
+--------+-----------+------------------+----------+--------------------+--------+----------+------------+--------------------+
|      37|         20|    Kristy Flowers|        14|pamela11@example.com|       6|2024-07-06|     Pending|6479 Thomas Trail...|
|      80|         38|     Keith Sanchez|         6|lisaholt@example.org|       6|2024-08-10|     Shipped|07201 Berry Passa...|
|      52|         70|       Frank Myers|        30|  nyoung@example.net|       4|2024-01-17|     Shipped|Unit 8916 Box 262...|
|      39|         38|    Samantha Gomez|        26|suarezjennifer@ex...|       4|2024-04-17|     Shipped|8091 Sellers Ferr...|
|      87|         79|     Brian Blevins|        20|carterluis@exampl...|       4|2024-01-08|   Cancelle

In [167]:
df.replace(0, 3.5, subset=["rating"]).show()

+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|      product_name| Price|         Description|Category|StockQuantity|       Brand|Rating|         Warehouse|
+------------------+------+--------------------+--------+-------------+------------+------+------------------+
|         Card Game|363.33|Whom those term b...|    Toys|           95|        Nerf|   2.6|        Emilyburgh|
|          Toy Boat|  79.6|Listen economy ge...|    Toys|           34|      Barbie|   2.8|  North Jamesmouth|
|     Action Figure|432.64|Lead back her local.|    Toys|            8|        Nerf|   4.6|  West Joshuahaven|
|      Rubik's Cube|215.39|Watch growth inve...|    Toys|         NULL|      Barbie|   3.5|   Mckenziechester|
|       Barbie Doll|  NULL|                NULL|    Toys|           38|   DC Comics|   1.2|     Catherineside|
|    Stuffed Animal|220.09|Computer little f...|    Toys|           76|      Chicco|   2.3|         Bryanport|
|